In [ ]:
import torch
import torchvision
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch._dynamo
torch._dynamo.config.suppress_errors = True
from torch import nn
from torchvision import transforms
try:
  from torchinfo import summary
except ModuleNotFoundError as e:
  print(f"{e}, Downloading..")
  !pip install torchinfo
  from torchinfo import summary

In [ ]:
from modules import model, train
model, train_transforms, test_transforms = model.create_model(10)
!mkdir "./data" "./data/train" "./data/test"
train_data = torchvision.datasets.INaturalist(root="./data/train",
                                              version="2021_train_mini",
                                              transform=train_transforms,
                                              download=True)
test_data = torchvision.datasets.INaturalist(root="./data/test",
                                             version="2021_valid",
                                             transform=test_transforms,
                                             download=True)

In [ ]:
import shutil

os.listdir("./data/train/2021_train_mini")[10:] == os.listdir("./data/test/2021_valid")[10:]

for i in os.listdir("./data/train/2021_train_mini")[10:]:
  shutil.rmtree("./data/train/2021_train_mini/"+i)

for i in os.listdir("./data/test/2021_valid")[10:]:
  shutil.rmtree("./data/test/2021_valid/"+i)

os.listdir("./data/train/2021_train_mini") == os.listdir("./data/test/2021_valid")

In [ ]:
from modules import model, train
model, train_transforms, test_transforms = model.create_model(10)

train_data = torchvision.datasets.ImageFolder(root="./data/train/2021_train_mini",
                                              transform=train_transforms)
test_data = torchvision.datasets.ImageFolder(root="./data/test/2021_valid",
                                             transform=test_transforms)

train_dataloader = torch.utils.data.DataLoader(dataset=train_data,
                                               batch_size=128,
                                               shuffle=True,
                                               num_workers=os.cpu_count(),
                                               pin_memory=True,
                                               generator=torch.Generator(device="cpu")
                                               )
test_dataloader = torch.utils.data.DataLoader(dataset=test_data,
                                              batch_size=128,
                                              shuffle=True,
                                              num_workers=os.cpu_count(),
                                              pin_memory=True,
                                              generator=torch.Generator(device="cpu"))


In [ ]:
summary(model=model,
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
import pandas as pd 
data = [i.split("_") for i in os.listdir("./data/train/2021_train_mini")]
cols = ["label","kingdom", "phylum", "class", "order", "family", "genus", "name"]
df = pd.DataFrame(data=data, columns=cols)
df.sort_values("label", inplace=True)
df.set_index("label", drop=True, inplace=True)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
torch.compile(model)
EPOCHS = 50

In [ ]:
val = next(iter(test_dataloader))


In [ ]:
import pandas as pd 
data = [i.split("_") for i in os.listdir("./data/train/2021_train_mini")]
cols = ["label","kingdom", "phylum", "class", "order", "family", "genus", "name"]
df = pd.DataFrame(data=data, columns=cols)
df.sort_values("label", inplace=True)
df.set_index("label", drop=True, inplace=True)

In [ ]:
import random
random_img = random.randint(0, 127)
model.eval()
with torch.inference_mode():
  img = val[0][random_img]
  img_converted = img.unsqueeze(dim=0)
  img_converted = model(img_converted.to(device))
  pred_label = torch.argmax(torch.softmax(img_converted, dim=1), dim=1)
  plt.imshow(img.cpu().permute(1, 2, 0));
  plt.title(f"Prediction Label: {df.iloc[pred_label.max()]} | True Label: {df.iloc[random_img]}")
  plt.axis("off");